In [1]:
import pandas as pd
import numpy as np

import ast


pd.set_option('display.max_rows', 100)

In [2]:
templates_table = pd.read_csv('../../DATA/RAW/register_uz_apr23/preprocessed/templates_dataset.csv')
reports_table = pd.read_csv('../../DATA/TRANSFORM/financial_statements/financial_reports_table_sample.csv')
statements_table = pd.read_csv('../../DATA/TRANSFORM/financial_statements/financial_statements_table.csv')

statements_table = statements_table[['financial_statement_id', 'entity_id', 'entity_ico', 'year', 'month']]

C:\Users\matus.rohal\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3433: DtypeWarning: Columns (12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
reports_table = reports_table.merge(statements_table, on = 'financial_statement_id', how = 'left')
statements_table = None

In [4]:
templates_table.query('template_id in [687, 699, 22, 21, 943]')[['template_id', 'template_name']].drop_duplicates()

,template_id,template_name
2526,21,Súvaha Úč POD 1-01
2673,22,Výkaz ziskov a strát Úč POD 2-01
4770,687,Úč MUJ
5263,699,Úč POD
7158,943,Výkaz vybraných údajov


In [5]:
def print_template(template_id):
    table = templates_table.query('template_id == @template_id')[['table_name', 'type', 'row', 'column', 'text']]
    columns = table.query('type == "column"')
    
    header_width = int(columns['column'].max())
    header_height = int(columns['row'].max())
    tables_headers = []
    for table_name in columns['table_name'].unique():
        print(table_name)
        table_header = []
        for row_number in range(1, header_height + 1):
            header_row = []
            for col_number in range(1, header_width + 1):
                try:
                    header_row.append(columns.query('table_name == @table_name and column == @col_number and row == @row_number').iloc[0,4])
                except IndexError:
                    header_row.append('-')
                    
            print(header_row)  
            table_header.append(header_row)
        tables_headers.append(table_header)
        
        display(table.query('type == "row"').query('table_name == @table_name')[['row', 'text']].sort_values('row').set_index('row').reset_index())
        table.query('type == "row"').query('table_name == @table_name')[['row', 'text']].sort_values('row').set_index('row').to_csv(f'statement_{template_id}_{table_name}.csv')
        print()


In [6]:
templates_codes = {
    'MUJ': 687,
    'POD': 699,
    'POD_1': 21,
    'POD_2': 22,
    'VVU': 943,
}

In [7]:
# for key, value in templates_codes.items():
#     print(key)
#     print_template(value)

print_template(templates_codes['MUJ'])
print_template(templates_codes['POD'])

Strana aktív
['Označenie', 'STRANA AKTÍV', 'Číslo riadku', 'Bežné účtovné obdobie', 'Bezprostredne predchádzajúce účtovné obdobie']
['a', 'b', 'c', 'Netto 1', 'Netto 2']


,row,text
0,1.0,SPOLU MAJETOK r. 02 + r. 14
1,2.0,Neobežný majetok r. 03 + r. 04 + r. 09
2,3.0,"Dlhodobý nehmotný majetok (012, 013, 014, 015,..."
3,4.0,Dlhodobý hmotný majetok súčet (r. 05 až r. 08)
4,5.0,"Pozemky a stavby (021, 031, 042A, 052A) - /081..."
5,6.0,Samostatné hnuteľné veci a súbory hnuteľných v...
6,7.0,"Ostatný dlhodobý hmotný majetok (025, 026, 029..."
7,8.0,Opravná položka k nadobudnutému majetku (+/- 0...
8,9.0,Dlhodobý finančný majetok súčet (r. 10 až r. 13)
9,10.0,"Podielové cenné papiere (061, 062, 063, 043A, ..."



Strana pasív
['Označenie', 'STRANA PASÍV', 'Číslo riadku', 'Bežné účtovné obdobie', 'Bezprostredne predchádzajúce účtovné obdobie']
['a', 'b', 'c', '3', '4']


,row,text
0,24.0,SPOLU VLASTNÉ IMANIE A ZÁVÄZKY r. 25 + r. 34
1,25.0,Vlastné imanie r. 26 + r. 29 + r. 30 + r. 31 +...
2,26.0,Základné imanie r. 27 + r. 28
3,27.0,Základné imanie a zmeny základného imania (411...
4,28.0,Pohľadávky za upísané vlastné imanie (/-/353)
5,29.0,"Kapitálové fondy (412, 413, 417, 418)"
6,30.0,"Fondy zo zisku (421, 422, 423, 427, 42X)"
7,31.0,"Oceňovacie rozdiely (+/- 415, 416)"
8,32.0,Nerozdelený zisk alebo neuhradená strata minul...
9,33.0,Výsledok hospodárenia za účtovné obdobie po zd...



Výkaz ziskov a strát
['Označenie', 'Text', 'Číslo riadku', 'Bežné účtovné obdobie', 'Bezprostredne predchádzajúce účtovné obdobie']
['a', 'b', 'c', '1', '2']


...

,row,text
0,1.0,SPOLU MAJETOK r. 02 + r. 33 + r. 74
1,2.0,Neobežný majetok r. 03 + r. 11 + r. 21
2,3.0,Dlhodobý nehmotný majetok súčet (r. 04 až r. 10)
3,4.0,"Aktivované náklady na vývoj (012) - /072, 091A/"
4,5.0,"Softvér (013) - /073, 091A/"
5,6.0,"Oceniteľné práva (014) - /074, 091A/"
6,7.0,"Goodwill (015) - /075, 091A/"
7,8.0,"Ostatný dlhodobý nehmotný majetok (019, 01X) -..."
8,9.0,Obstarávaný dlhodobý nehmotný majetok (041) - ...
9,10.0,Poskytnuté preddavky na dlhodobý nehmotný maje...



Strana pasív
['Označenie', 'STRANA PASÍV', 'Číslo riadku', 'Bežné účtovné obdobie', 'Bezprostredne predchádzajúce účtovné obdobie', '-', '-']
['a', 'b', 'c', '4', '5', '-', '-']
['-', '-', '-', '-', '-', '-', '-']


,row,text
0,79.0,SPOLU VLASTNÉ IMANIE A ZÁVÄZKY r. 80 + r. 101 ...
1,80.0,Vlastné imanie r. 81 + r. 85 + r. 86 + r. 87 +...
2,81.0,Základné imanie súčet (r. 82 až r. 84)
3,82.0,Základné imanie (411 alebo +/- 491)
4,83.0,Zmena základného imania +/- 419
5,84.0,Pohľadávky za upísané vlastné imanie (/-/353)
6,85.0,Emisné ážio (412)
7,86.0,Ostatné kapitálové fondy (413)
8,87.0,Zákonné rezervné fondy r. 88 + r. 89
9,88.0,Zákonný rezervný fond a nedeliteľný fond (417A...



Výkaz ziskov a strát
['Označenie', 'Text', 'Číslo riadku', 'Skutočnosť', '-', '-', '-']
['-', '-', '-', 'Bežné účtovné obdobie', 'Bezprostredne predchádzajúce účtovné obdobie', '-', '-']
['a', 'b', 'c', '1', '2', '-', '-']


,row,text
0,1.0,Čistý obrat (časť účt. tr. 6 podľa zákona)
1,2.0,Výnosy z hospodárskej činnosti spolu súčet (r....
2,3.0,"Tržby z predaja tovaru (604, 607)"
3,4.0,Tržby z predaja vlastných výrobkov (601)
4,5.0,"Tržby z predaja služieb (602, 606)"
5,6.0,Zmeny stavu vnútroorganizačných zásob (+/-) (ú...
6,7.0,Aktivácia (účtová skupina 62)
7,8.0,"Tržby z predaja dlhodobého nehmotného majetku,..."
8,9.0,"Ostatné výnosy z hospodárskej činnosti (644, 6..."
9,10.0,Náklady na hospodársku činnosť spolu r. 11 + r...


In [8]:

class ReportContentParser:

    def parse(self, template_id, content_string):

        content = ast.literal_eval(content_string)
        
        display('title page:', content.get('titulnaStrana'))

        tables_content = content.get('tabulky')
        if tables_content is None:
            print('DOES NOT CONTAIN TABLES')
            return

        values = []
        if template_id == 687: values = self.__parse_MUJ(tables_content)
        if template_id == 699: values = self.__parse_POD(tables_content)         
        if template_id == 21: values = self.__parse_POD_1(tables_content)        
        if template_id == 22: values = self.__parse_POD_2(tables_content)
        print(values)
    
    def __parse_title_page(self, title_page_content): return None
    
    def __parse_MUJ(self, tables_content):
        # 687

        assets = tables_content[0]['data']
        assets = np.reshape(np.array(assets), (len(assets)//2, 2))[:, 0]
        assets[assets=='']='0'
        assets = assets.astype(float)

        liab = tables_content[1]['data']
        liab = np.reshape(np.array(liab), (len(liab)//2, 2))[:, 0]
        liab[liab=='']='0'
        liab = liab.astype(float)

        balance_sheet = np.concatenate([assets, liab])

        income = tables_content[2]['data']
        income = np.reshape(np.array(income), (len(income)//2, 2))[:, 0]
        income[income=='']='0'
        income = income.astype(float)

        profit_and_loss = income

        total_assets = balance_sheet[0] # total_assets
        non_current_assets = balance_sheet[1] # non_current_assets
        intangible_assets = balance_sheet[2] # intangible_assets
        total_non_current_tangible_assets = balance_sheet[3] # total_non_current_tangible_assets
        land_and_buildings = balance_sheet[4] # land_and_buildings
        separate_movable_assets_and_sets_of_movable_assets = balance_sheet[5] # separate_movable_assets_and_sets_of_movable_assets
        other_long_term_tangible_assets = balance_sheet[6] # other_long_term_tangible_assets
        adjustment_item_for_acquired_assets = balance_sheet[7] # adjustment_item_for_acquired_assets
        total_long_term_financial_assets = balance_sheet[8] # total_long_term_financial_assets
        bank_balances_with_maturity_longer_than_one_year = balance_sheet[11] # bank_balances_with_maturity_longer_than_one_year
        current_assets = balance_sheet[13] # current_assets
        inventories = balance_sheet[14] # inventories
        long_term_receivables = balance_sheet[15] # long_term_receivables
        total_short_term_receivables = balance_sheet[16] # total_short_term_receivables
        trade_receivables = balance_sheet[17] # trade_receivables
        social_security_tax_receivables_and_grants = balance_sheet[18] # social_security_tax_receivables_and_grants
        financial_assets = balance_sheet[20] # financial_assets
        other_financial_accounts = balance_sheet[22] # other_financial_accounts
        cash_and_bank_balances = balance_sheet[21] # cash_and_bank_balances


        total_owners_equity_and_liabilities=balance_sheet[23] # total_owners_equity_and_liabilities
        owners_equity=balance_sheet[24] # owners_equity
        share_capital=balance_sheet[25] # share_capital
        share_capital_and_changes_in_share_capital=balance_sheet[26] # share_capital_and_changes_in_share_capital
        claims_for_subscribed_owners_equity=balance_sheet[27] # claims_for_subscribed_owners_equity
        capital_funds=balance_sheet[28]+balance_sheet[29] # capital_funds


        total_operating_revenues=profit_and_loss[0] # total_operating_revenues
        sales_revenue=profit_and_loss[1] # sales_revenue
        revenue_from_the_sale_of_own_products_and_services=profit_and_loss[2] # revenue_from_the_sale_of_own_products_and_services
        change_in_intra_organizational_inventories=profit_and_loss[3] # change_in_intra_organizational_inventories
        activation=profit_and_loss[4] # activation
        revenue_from_the_sale_of_long_term_intangible_assets_long_term_tangible_assets_and_materials=profit_and_loss[5] # revenue_from_the_sale_of_long_term_intangible_assets_long_term_tangible_assets_and_materials
        other_revenues_from_operating_activities=profit_and_loss[6] # other_revenues_from_operating_activities
        total_operating_costs=profit_and_loss[7] # total_operating_costs
        costs_incurred_to_acquire_sold_goods=profit_and_loss[8] # costs_incurred_to_acquire_sold_goods
        consumption_of_materials_energy_and_other_non_inventory_supplies=profit_and_loss[9] # consumption_of_materials_energy_and_other_non_inventory_supplies
        services=profit_and_loss[10] # services
        personnel_costs=profit_and_loss[11] # personnel_costs
        taxes_and_fees=profit_and_loss[12] # taxes_and_fees
        depreciation_and_adjustments_to_long_term_intangible_assets_and_long_term_tangible_assets=profit_and_loss[13] # depreciation_and_adjustments_to_long_term_intangible_assets_and_long_term_tangible_assets
        remaining_cost_of_sold_long_term_assets_and_materials=profit_and_loss[14] # remaining_cost_of_sold_long_term_assets_and_materials
        adjustments_to_receivables=profit_and_loss[15] # adjustments_to_receivables
        other_costs_of_operating_activities=profit_and_loss[16] # other_costs_of_operating_activities
        operating_result_from_operating_activities=profit_and_loss[17] # operating_result_from_operating_activities
        value_added=profit_and_loss[18] # value_added
        total_financial_revenues=profit_and_loss[19] # total_financial_revenues
        revenue_from_the_sale_of_securities_and_shares=profit_and_loss[20] # revenue_from_the_sale_of_securities_and_shares
        revenue_from_long_term_financial_assets=profit_and_loss[21] # revenue_from_long_term_financial_assets
        revenue_from_short_term_financial_assets=profit_and_loss[22] # revenue_from_short_term_financial_assets
        interest_income=profit_and_loss[23] # interest_income
        exchange_gains=profit_and_loss[24] # exchange_gains
        other_revenues_from_financial_activities=profit_and_loss[25] # other_revenues_from_financial_activities
        total_financial_costs=profit_and_loss[26] # total_financial_costs
        sale_of_securities_and_shares=profit_and_loss[27] # sale_of_securities_and_shares
        costs_of_short_term_financial_assets=profit_and_loss[28] # costs_of_short_term_financial_assets
        adjustments_to_financial_assets=profit_and_loss[29] # adjustments_to_financial_assets
        interest_expenses=profit_and_loss[30] # interest_expenses
        exchange_losses=profit_and_loss[31] # exchange_losses
        other_costs_of_financial_activities=profit_and_loss[32] # other_costs_of_financial_activities
        financial_result=profit_and_loss[33] # financial_result
        result_of_operations_for_the_reporting_period_before_tax=profit_and_loss[34] # result_of_operations_for_the_reporting_period_before_tax
        income_tax=profit_and_loss[35] # income_tax
        transfer_of_shares_in_the_result_to_shareholders=profit_and_loss[36] # transfer_of_shares_in_the_result_to_shareholders
        result_of_operations_for_the_reporting_period_after_tax=profit_and_loss[37] # result_of_operations_for_the_reporting_period_after_tax


        values = [
            total_assets,
            non_current_assets,
            intangible_assets,
            total_non_current_tangible_assets,
            land_and_buildings,
            separate_movable_assets_and_sets_of_movable_assets,
            other_long_term_tangible_assets,
            adjustment_item_for_acquired_assets,
            total_long_term_financial_assets,
            bank_balances_with_maturity_longer_than_one_year,
            current_assets,
            inventories,
            long_term_receivables,
            total_short_term_receivables,
            trade_receivables,
            social_security_tax_receivables_and_grants,
            financial_assets,
            other_financial_accounts,
            cash_and_bank_balances,
            total_owners_equity_and_liabilities,
            owners_equity,
            share_capital,
            share_capital_and_changes_in_share_capital,
            claims_for_subscribed_owners_equity,
            capital_funds,
            total_operating_revenues,
            sales_revenue,
            revenue_from_the_sale_of_own_products_and_services,
            change_in_intra_organizational_inventories,
            activation,
            revenue_from_the_sale_of_long_term_intangible_assets_long_term_tangible_assets_and_materials,
            other_revenues_from_operating_activities,
            total_operating_costs,
            costs_incurred_to_acquire_sold_goods,
            consumption_of_materials_energy_and_other_non_inventory_supplies,
            services,
            personnel_costs,
            taxes_and_fees,
            depreciation_and_adjustments_to_long_term_intangible_assets_and_long_term_tangible_assets,
            remaining_cost_of_sold_long_term_assets_and_materials,
            adjustments_to_receivables,
            other_costs_of_operating_activities,
            operating_result_from_operating_activities,
            value_added,
            total_financial_revenues,
            revenue_from_the_sale_of_securities_and_shares,
            revenue_from_long_term_financial_assets,
            revenue_from_short_term_financial_assets,
            interest_income,
            exchange_gains,
            other_revenues_from_financial_activities,
            total_financial_costs,
            sale_of_securities_and_shares,
            costs_of_short_term_financial_assets,
            adjustments_to_financial_assets,
            interest_expenses,
            exchange_losses,
            other_costs_of_financial_activities,
            financial_result,
            result_of_operations_for_the_reporting_period_before_tax,
            income_tax,
            transfer_of_shares_in_the_result_to_shareholders,
            result_of_operations_for_the_reporting_period_after_tax
        ]

        header = [
            "total_assets",
            "non_current_assets",
            "intangible_assets",
            "total_non_current_tangible_assets",
            "land_and_buildings",
            "separate_movable_assets_and_sets_of_movable_assets",
            "other_long_term_tangible_assets",
            "adjustment_item_for_acquired_assets",
            "total_long_term_financial_assets",
            "bank_balances_with_maturity_longer_than_one_year",
            "current_assets",
            "inventories",
            "long_term_receivables",
            "total_short_term_receivables",
            "trade_receivables",
            "social_security_tax_receivables_and_grants",
            "financial_assets",
            "other_financial_accounts",
            "cash_and_bank_balances",
            "total_owners_equity_and_liabilities",
            "owners_equity",
            "share_capital",
            "share_capital_and_changes_in_share_capital",
            "claims_for_subscribed_owners_equity",
            "capital_funds",
            "total_operating_revenues",
            "sales_revenue",
            "revenue_from_the_sale_of_own_products_and_services",
            "change_in_intra_organizational_inventories",
            "activation",
            "revenue_from_the_sale_of_long_term_intangible_assets_long_term_tangible_assets_and_materials",
            "other_revenues_from_operating_activities",
            "total_operating_costs",
            "costs_incurred_to_acquire_sold_goods",
            "consumption_of_materials_energy_and_other_non_inventory_supplies",
            "services",
            "personnel_costs",
            "taxes_and_fees",
            "depreciation_and_adjustments_to_long_term_intangible_assets_and_long_term_tangible_assets",
            "remaining_cost_of_sold_long_term_assets_and_materials",
            "adjustments_to_receivables",
            "other_costs_of_operating_activities",
            "operating_result_from_operating_activities",
            "value_added",
            "total_financial_revenues",
            "revenue_from_the_sale_of_securities_and_shares",
            "revenue_from_long_term_financial_assets",
            "revenue_from_short_term_financial_assets",
            "interest_income",
            "exchange_gains",
            "other_revenues_from_financial_activities",
            "total_financial_costs",
            "sale_of_securities_and_shares",
            "costs_of_short_term_financial_assets",
            "adjustments_to_financial_assets",
            "interest_expenses",
            "exchange_losses",
            "other_costs_of_financial_activities",
            "financial_result",
            "result_of_operations_for_the_reporting_period_before_tax",
            "income_tax",
            "transfer_of_shares_in_the_result_to_shareholders",
            "result_of_operations_for_the_reporting_period_after_tax"
        ]
        

        df = pd.DataFrame([values], columns = header)
        return df
    
    def __parse_POD(self, tables_content):
        # 699
        assets = tables_content[0]['data']
        assets = np.reshape(np.array(assets), (len(assets)//4, 4))[:, 2]
        assets[assets=='']='0'
        assets = assets.astype(float)

        liab = tables_content[1]['data']
        liab = np.reshape(np.array(liab), (len(liab)//2, 2))[:, 0]
        liab[liab=='']='0'
        liab = liab.astype(float)

        balance_sheet = np.concatenate([assets, liab])

        income = tables_content[2]['data']
        income = np.reshape(np.array(income), (len(income)//2, 2))[:, 0]
        income[income=='']='0'
        income = income.astype(float)

        profit_and_loss = income

        total_assets=balance_sheet[0] # total_assets
        non_current_assets=balance_sheet[1] # non_current_assets
        intangible_assets=balance_sheet[2] # intangible_assets
        total_non_current_tangible_assets=balance_sheet[10] # total_non_current_tangible_assets
        land_and_buildings=balance_sheet[11]+balance_sheet[12] # land_and_buildings
        separate_movable_assets_and_sets_of_movable_assets=balance_sheet[13] # separate_movable_assets_and_sets_of_movable_assets
        other_long_term_tangible_assets=balance_sheet[16] # other_long_term_tangible_assets
        adjustment_item_for_acquired_assets=balance_sheet[19] # adjustment_item_for_acquired_assets
        total_long_term_financial_assets=balance_sheet[20] # total_long_term_financial_assets
        bank_balances_with_maturity_longer_than_one_year=balance_sheet[29] # bank_balances_with_maturity_longer_than_one_year
        current_assets=balance_sheet[32] # current_assets
        inventories=balance_sheet[33] # inventories
        long_term_receivables=balance_sheet[40] # long_term_receivables
        total_short_term_receivables=balance_sheet[52] # total_short_term_receivables
        trade_receivables=balance_sheet[53] # trade_receivables
        social_security_tax_receivables_and_grants=balance_sheet[61]+balance_sheet[62] # social_security_tax_receivables_and_grants
        financial_assets=balance_sheet[65]+balance_sheet[70] # financial_assets
        other_financial_accounts=balance_sheet[65] # other_financial_accounts
        cash_and_bank_balances=balance_sheet[70] # cash_and_bank_balances
        total_owners_equity_and_liabilities=balance_sheet[78] # total_owners_equity_and_liabilities
        owners_equity=balance_sheet[79] # owners_equity
        share_capital=balance_sheet[80] # share_capital
        share_capital_and_changes_in_share_capital=balance_sheet[81]+balance_sheet[82] # share_capital_and_changes_in_share_capital
        claims_for_subscribed_owners_equity=balance_sheet[83] # claims_for_subscribed_owners_equity
        capital_funds=balance_sheet[84]+balance_sheet[85]+balance_sheet[86]+balance_sheet[89]+balance_sheet[92] # capital_funds
        valuation_differences=balance_sheet[94]+balance_sheet[96] # valuation_differences
        undivided_profit_or_unpaid_loss_from_previous_years=balance_sheet[96] # undivided_profit_or_unpaid_loss_from_previous_years
        profit_for_the_reporting_period_after_tax=balance_sheet[99] # profit_for_the_reporting_period_after_tax
        liabilities=balance_sheet[100] # liabilities
        long_term_liabilities_excluding_reserves_and_loans=balance_sheet[101] # long_term_liabilities_excluding_reserves_and_loans
        long_term_reserves=balance_sheet[117] # long_term_reserves
        long_term_bank_loans=balance_sheet[120] # long_term_bank_loans
        total_short_term_liabilities_excluding_reserves_loans_and_advances=balance_sheet[121] # total_short_term_liabilities_excluding_reserves_loans_and_advances
        short_term_trade_liabilities=balance_sheet[122] # short_term_trade_liabilities
        liabilities_to_employees_and_social_insurance=balance_sheet[130]+balance_sheet[131] # liabilities_to_employees_and_social_insurance
        tax_liabilities_and_grants=balance_sheet[132] # tax_liabilities_and_grants
        short_term_reserves=balance_sheet[135] # short_term_reserves
        current_bank_loans=balance_sheet[138] # current_bank_loans

        total_operating_revenues=profit_and_loss[1] # total_operating_revenues
        sales_revenue=profit_and_loss[2] # sales_revenue
        revenue_from_the_sale_of_own_products_and_services=profit_and_loss[3]+profit_and_loss[4] # revenue_from_the_sale_of_own_products_and_services

        change_in_intra_organizational_inventories=profit_and_loss[5] # change_in_intra_organizational_inventories
        activation=profit_and_loss[6] # activation
        revenue_from_the_sale_of_long_term_intangible_assets_long_term_tangible_assets_and_materials=profit_and_loss[7] # revenue_from_the_sale_of_long_term_intangible_assets_long_term_tangible_assets_and_materials
        other_revenues_from_operating_activities=profit_and_loss[8] # other_revenues_from_operating_activities
        total_operating_costs=profit_and_loss[9] # total_operating_costs
        costs_incurred_to_acquire_sold_goods=profit_and_loss[10] # costs_incurred_to_acquire_sold_goods
        consumption_of_materials_energy_and_other_non_inventory_supplies=profit_and_loss[11] # consumption_of_materials_energy_and_other_non_inventory_supplies
        services=profit_and_loss[13] # services
        personnel_costs=profit_and_loss[14] # personnel_costs
        taxes_and_fees=profit_and_loss[19] # taxes_and_fees
        depreciation_and_adjustments_to_long_term_intangible_assets_and_long_term_tangible_assets=profit_and_loss[20] # depreciation_and_adjustments_to_long_term_intangible_assets_and_long_term_tangible_assets
        remaining_cost_of_sold_long_term_assets_and_materials=profit_and_loss[23] # remaining_cost_of_sold_long_term_assets_and_materials
        adjustments_to_receivables=profit_and_loss[24] # adjustments_to_receivables
        other_costs_of_operating_activities=profit_and_loss[25] # other_costs_of_operating_activities
        operating_result_from_operating_activities=profit_and_loss[26] # operating_result_from_operating_activities
        value_added=profit_and_loss[27] # value_added
        total_financial_revenues=profit_and_loss[28] # total_financial_revenues
        revenue_from_the_sale_of_securities_and_shares=profit_and_loss[29] # revenue_from_the_sale_of_securities_and_shares
        revenue_from_long_term_financial_assets=profit_and_loss[30] # revenue_from_long_term_financial_assets
        revenue_from_short_term_financial_assets=profit_and_loss[34] # revenue_from_short_term_financial_assets
        interest_income=profit_and_loss[38] # interest_income
        exchange_gains=profit_and_loss[41] # exchange_gains
        other_revenues_from_financial_activities=profit_and_loss[43] # other_revenues_from_financial_activities
        total_financial_costs=profit_and_loss[44] # total_financial_costs
        sale_of_securities_and_shares=profit_and_loss[45] # sale_of_securities_and_shares
        costs_of_short_term_financial_assets=profit_and_loss[46] # costs_of_short_term_financial_assets
        adjustments_to_financial_assets=profit_and_loss[47] # adjustments_to_financial_assets
        interest_expenses=profit_and_loss[48] # interest_expenses
        exchange_losses=profit_and_loss[51] # exchange_losses
        other_costs_of_financial_activities=profit_and_loss[53] # other_costs_of_financial_activities
        financial_result=profit_and_loss[54] # financial_result
        result_of_operations_for_the_reporting_period_before_tax=profit_and_loss[55] # result_of_operations_for_the_reporting_period_before_tax
        income_tax=profit_and_loss[56] # income_tax
        transfer_of_shares_in_the_result_to_shareholders=profit_and_loss[59] # transfer_of_shares_in_the_result_to_shareholders
        result_of_operations_for_the_reporting_period_after_tax=profit_and_loss[60] # result_of_operations_for_the_reporting_period_after_tax


        values = [
            total_assets,
            non_current_assets,
            intangible_assets,
            total_non_current_tangible_assets,
            land_and_buildings,
            separate_movable_assets_and_sets_of_movable_assets,
            other_long_term_tangible_assets,
            adjustment_item_for_acquired_assets,
            total_long_term_financial_assets,
            bank_balances_with_maturity_longer_than_one_year,
            current_assets,
            inventories,
            long_term_receivables,
            total_short_term_receivables,
            trade_receivables,
            social_security_tax_receivables_and_grants,
            financial_assets,
            other_financial_accounts,
            cash_and_bank_balances,
            total_owners_equity_and_liabilities,
            owners_equity,
            share_capital,
            share_capital_and_changes_in_share_capital,
            claims_for_subscribed_owners_equity,
            capital_funds,
            valuation_differences,
            undivided_profit_or_unpaid_loss_from_previous_years,
            profit_for_the_reporting_period_after_tax,
            liabilities,
            long_term_liabilities_excluding_reserves_and_loans,
            long_term_reserves,
            long_term_bank_loans,
            total_short_term_liabilities_excluding_reserves_loans_and_advances,
            short_term_trade_liabilities,
            liabilities_to_employees_and_social_insurance,
            tax_liabilities_and_grants,
            short_term_reserves,
            current_bank_loans,
            total_operating_revenues,
            sales_revenue,
            revenue_from_the_sale_of_own_products_and_services,
            change_in_intra_organizational_inventories,
            activation,
            revenue_from_the_sale_of_long_term_intangible_assets_long_term_tangible_assets_and_materials,
            other_revenues_from_operating_activities,
            total_operating_costs,
            costs_incurred_to_acquire_sold_goods,
            consumption_of_materials_energy_and_other_non_inventory_supplies,
            services,
            personnel_costs,
            taxes_and_fees,
            depreciation_and_adjustments_to_long_term_intangible_assets_and_long_term_tangible_assets,
            remaining_cost_of_sold_long_term_assets_and_materials,
            adjustments_to_receivables,
            other_costs_of_operating_activities,
            operating_result_from_operating_activities,
            value_added,
            total_financial_revenues,
            revenue_from_the_sale_of_securities_and_shares,
            revenue_from_long_term_financial_assets,
            revenue_from_short_term_financial_assets,
            interest_income,
            exchange_gains,
            other_revenues_from_financial_activities,
            total_financial_costs,
            sale_of_securities_and_shares,
            costs_of_short_term_financial_assets,
            adjustments_to_financial_assets,
            interest_expenses,
            exchange_losses,
            other_costs_of_financial_activities,
            financial_result,
            result_of_operations_for_the_reporting_period_before_tax,
            income_tax,
            transfer_of_shares_in_the_result_to_shareholders,
            result_of_operations_for_the_reporting_period_after_tax,
        ]

        header = [
            "total_assets",
            "non_current_assets",
            "intangible_assets",
            "total_non_current_tangible_assets",
            "land_and_buildings",
            "separate_movable_assets_and_sets_of_movable_assets",
            "other_long_term_tangible_assets",
            "adjustment_item_for_acquired_assets",
            "total_long_term_financial_assets",
            "bank_balances_with_maturity_longer_than_one_year",
            "current_assets",
            "inventories",
            "long_term_receivables",
            "total_short_term_receivables",
            "trade_receivables",
            "social_security_tax_receivables_and_grants",
            "financial_assets",
            "other_financial_accounts",
            "cash_and_bank_balances",
            "total_owners_equity_and_liabilities",
            "owners_equity",
            "share_capital",
            "share_capital_and_changes_in_share_capital",
            "claims_for_subscribed_owners_equity",
            "capital_funds",
            "valuation_differences",
            "undivided_profit_or_unpaid_loss_from_previous_years",
            "profit_for_the_reporting_period_after_tax",
            "liabilities",
            "long_term_liabilities_excluding_reserves_and_loans",
            "long_term_reserves",
            "long_term_bank_loans",
            "total_short_term_liabilities_excluding_reserves_loans_and_advances",
            "short_term_trade_liabilities",
            "liabilities_to_employees_and_social_insurance",
            "tax_liabilities_and_grants",
            "short_term_reserves",
            "current_bank_loans",
            "total_operating_revenues",
            "sales_revenue",
            "revenue_from_the_sale_of_own_products_and_services",
            "change_in_intra_organizational_inventories",
            "activation",
            "revenue_from_the_sale_of_long_term_intangible_assets_long_term_tangible_assets_and_materials",
            "other_revenues_from_operating_activities",
            "total_operating_costs",
            "costs_incurred_to_acquire_sold_goods",
            "consumption_of_materials_energy_and_other_non_inventory_supplies",
            "services",
            "personnel_costs",
            "taxes_and_fees",
            "depreciation_and_adjustments_to_long_term_intangible_assets_and_long_term_tangible_assets",
            "remaining_cost_of_sold_long_term_assets_and_materials",
            "adjustments_to_receivables",
            "other_costs_of_operating_activities",
            "operating_result_from_operating_activities",
            "value_added",
            "total_financial_revenues",
            "revenue_from_the_sale_of_securities_and_shares",
            "revenue_from_long_term_financial_assets",
            "revenue_from_short_term_financial_assets",
            "interest_income",
            "exchange_gains",
            "other_revenues_from_financial_activities",
            "total_financial_costs",
            "sale_of_securities_and_shares",
            "costs_of_short_term_financial_assets",
            "adjustments_to_financial_assets",
            "interest_expenses",
            "exchange_losses",
            "other_costs_of_financial_activities",
            "financial_result",
            "result_of_operations_for_the_reporting_period_before_tax",
            "income_tax",
            "transfer_of_shares_in_the_result_to_shareholders",
            "result_of_operations_for_the_reporting_period_after_tax"
        ]

        df = pd.DataFrame([values], columns = header)
        return df        


    def __parse_POD_1(self, tables_content):
        # 21
        for table in tables_content:
            name = table.get('nazov')
            print(name)            
            data = table.get('data')
            if data is None:
                print('DOES NOT CONTAIN DATA')
            print(data)
        return pd.DataFrame()
    
    def __parse_POD_2(self, tables_content):
        # 22
        for table in tables_content:
            name = table.get('nazov')
            print(name)
            data = table.get('data')
            if data is None:
                print('DOES NOT CONTAIN DATA')
            print(data)
        return pd.DataFrame()


In [9]:
# Original varsion

# class ReportContentParser:

#     def parse(self, template_id, content_string):

#         content = ast.literal_eval(content_string)
        
#         display('title page:', content.get('titulnaStrana'))

#         tables_content = content.get('tabulky')
#         if tables_content is None:
#             print('DOES NOT CONTAIN TABLES')
#             return

#         values = []
#         if template_id == 687: values = self.__parse_MUJ(tables_content)
#         if template_id == 699: values = self.__parse_POD(tables_content)         
#         if template_id == 21: values = self.__parse_POD_1(tables_content)        
#         if template_id == 22: values = self.__parse_POD_2(tables_content)
#         print(values)
    
#     def __parse_title_page(self, title_page_content): return None
    
#     def __parse_MUJ(self, tables_content):
#         # 687

#         assets = tables_content[0]['data']
#         assets = np.reshape(np.array(assets), (len(assets)//2, 2))[:, 0]
#         assets[assets=='']='0'
#         assets = assets.astype(float)

#         liab = tables_content[1]['data']
#         liab = np.reshape(np.array(liab), (len(liab)//2, 2))[:, 0]
#         liab[liab=='']='0'
#         liab = liab.astype(float)

#         balance_sheet = np.concatenate([assets, liab])

#         income = tables_content[2]['data']
#         income = np.reshape(np.array(income), (len(income)//2, 2))[:, 0]
#         income[income=='']='0'
#         income = income.astype(float)

#         profit_and_loss = income

#         # assets
#         ca = balance_sheet[14]+balance_sheet[16]+balance_sheet[20]
#         ta = balance_sheet[0]
#         cash = balance_sheet[20]

#         # liablities
#         eq = balance_sheet[24]
#         cl = balance_sheet[37] + balance_sheet[44] + balance_sheet[43]
#         ncl = balance_sheet[34] + balance_sheet[36]
#         tl = cl + ncl

#         # profit and loss
#         cf = profit_and_loss[37] + profit_and_loss[13]
#         sal = profit_and_loss[1] + profit_and_loss[2] + profit_and_loss[5] + profit_and_loss[20]
#         eat = profit_and_loss[37]
#         ebit = profit_and_loss[30] + profit_and_loss[35] + profit_and_loss[37]

#         values = [ca, ta, eq, cl, ncl, tl, cash, cf, sal, eat, ebit]
#         display(pd.DataFrame([values], columns = ['ca', 'ta', 'eq', 'cl', 'ncl', 'tl', 'cash', 'cf', 'sal', 'eat', 'ebit']))
#         return values

    
#     def __parse_POD(self, tables_content):
#         # 699
#         assets = tables_content[0]['data']
#         assets = np.reshape(np.array(assets), (len(assets)//4, 4))[:, 2]
#         assets[assets=='']='0'
#         assets = assets.astype(float)

#         liab = tables_content[1]['data']
#         liab = np.reshape(np.array(liab), (len(liab)//2, 2))[:, 0]
#         liab[liab=='']='0'
#         liab = liab.astype(float)

#         balance_sheet = np.concatenate([assets, liab])

#         income = tables_content[2]['data']
#         income = np.reshape(np.array(income), (len(income)//2, 2))[:, 0]
#         income[income=='']='0'
#         income = income.astype(float)

#         profit_and_loss = income

#         # assets
#         ca = balance_sheet[70] + balance_sheet[52] + balance_sheet[33]
 
#         cc = balance_sheet[52]
#         ta = balance_sheet[0]
#         cash = balance_sheet[70]
        
#         # liablities
#         eq = balance_sheet[79]
#         tl = balance_sheet[100]
#         cl = balance_sheet[121]
        
#         # profit and loss
#         cf = profit_and_loss[60] + profit_and_loss[21]
#         sal = profit_and_loss[2] + profit_and_loss[3] + profit_and_loss[4] + profit_and_loss[7] + profit_and_loss[29]
#         eat = profit_and_loss[60]
#         ebit = profit_and_loss[26] - profit_and_loss[7] + profit_and_loss[23] + profit_and_loss[12] + profit_and_loss[22] + profit_and_loss[24] - profit_and_loss[8]
#         ebt = profit_and_loss[55]
#         it = profit_and_loss[38] + profit_and_loss[48]
#         stock = balance_sheet[33]
#         yie = profit_and_loss[1] + profit_and_loss[28]
#         cost = profit_and_loss[9] + profit_and_loss[44]

#         values = [ca, cc, ta, eq, cl, tl - cl, tl, cash, cf, sal, eat, ebit, ebt, stock, cost, it, yie]
#         display(pd.DataFrame([values], columns = ['ca', 'cc', 'ta', 'eq', 'cl', 'ncl', 'tl', 'cash', 'cf', 'sal', 'eat', 'ebit', 'ebt', 'stock', 'cost', 'it', 'yie']))
#         return values
    
#     def __parse_POD_1(self, tables_content):
#         # 21
#         for table in tables_content:
#             name = table.get('nazov')
#             print(name)            
#             data = table.get('data')
#             if data is None:
#                 print('DOES NOT CONTAIN DATA')
#             print(data)
#         return []
    
#     def __parse_POD_2(self, tables_content):
#         # 22
#         for table in tables_content:
#             name = table.get('nazov')
#             print(name)
#             data = table.get('data')
#             if data is None:
#                 print('DOES NOT CONTAIN DATA')
#             print(data)
#         return []


In [10]:
def get_random_report(query_string = None):
    if query_string is None:
        report = reports_table.sample(1).iloc[0, :]
    else:
        report = reports_table.query(query_string).sample(1).iloc[0, :]
    
    print('ICO', int(report.entity_ico))
    print(f'https://www.registeruz.sk/cruz-public/domain/accountingentity/show/{(report.entity_id)}')

    print()
    print(f'{report.month}/{report.year}')

    parser = ReportContentParser()
    parser.parse(report.template_id, report.content)



In [11]:
for template_id in reports_table['template_id'].unique():
    print('template:', template_id)
    qstring = f'template_id == {template_id}'
    get_random_report(qstring)
    print()

template: 699.0
ICO 35921510
https://www.registeruz.sk/cruz-public/domain/accountingentity/show/75066

1/2016


'title page:'

{'nazovUctovnejJednotky': 'ATREA SK s.r.o.',
 'ico': '35921510',
 'dic': '2021949501',
 'typUctovnejJednotky': 'malá',
 'adresa': {'ulica': 'DRUŽSTEVNÁ',
  'cislo': '2',
  'psc': '945 01',
  'mesto': 'KOMÁRNO'},
 'skNace': '46180',
 'typZavierky': 'Riadna',
 'obdobieOd': '2016-01',
 'obdobieDo': '2016-12',
 'predchadzajuceObdobieOd': '2015-01',
 'predchadzajuceObdobieDo': '2015-12',
 'datumZostavenia': '2017-06-15',
 'oznacenieObchodnehoRegistra': 'Okresný súd Nitra, Obchodný register 16267/N'}

   total_assets  non_current_assets  intangible_assets  \
0      720032.0             35714.0                0.0   

   total_non_current_tangible_assets  land_and_buildings  \
0                            35714.0                 0.0   

   separate_movable_assets_and_sets_of_movable_assets  \
0                                            35714.0    

   other_long_term_tangible_assets  adjustment_item_for_acquired_assets  \
0                              0.0                                  0.0   

   total_long_term_financial_assets  \
0                               0.0   

   bank_balances_with_maturity_longer_than_one_year  ...  \
0                                               0.0  ...   

   costs_of_short_term_financial_assets  adjustments_to_financial_assets  \
0                                   0.0                              0.0   

   interest_expenses  exchange_losses  other_costs_of_financial_activities  \
0                0.0            217.0                            

'title page:'

...

{'nazovUctovnejJednotky': 'UKR Consult, s.r.o.',
 'ico': '45882932',
 'dic': '2023145905',
 'adresa': {'ulica': 'ZÁMOCKÁ',
  'cislo': '30',
  'psc': '81101',
  'mesto': 'BRATISLAVA'},
 'skNace': '70220',
 'typZavierky': 'Riadna',
 'obdobieOd': '2013-01',
 'obdobieDo': '2013-12',
 'predchadzajuceObdobieOd': '2012-01',
 'predchadzajuceObdobieDo': '2012-12',
 'datumZostavenia': '2014-01-03'}

{'sk': 'Výkaz ziskov a strát'}
['', '', '', '', '', '', '27000', '142000', '27000', '142000', '', '', '', '', '8341', '16080', '', '852', '8341', '15228', '18659', '125920', '2545', '75826', '', '56054', '1892', '', '653', '19436', '', '336', '', '211', '', '', '', '', '', '', '', '', '', '', '', '23', '', '', '', '', '16114', '49860', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '1', '1', '', '', '', '', '', '', '', '', '68', '72', '', '', '', '', '-67', '-71', '16047', '49789', '3691', '8801', '3691', '8801', '', '', '12356', '40988', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '16047', '49789', '', '', '12356', '40988']
Empty DataFrame
Columns: []
Index: []

template: 21.0
ICO 46679855
https://www.registeruz.sk/cruz-public/domain/accountingentity/show/1233112

1/2013


'title page:'

{'nazovUctovnejJednotky': 'SK Forklifts and Rackings, s.r.o.',
 'ico': '46679855',
 'dic': '2023535910',
 'adresa': {'ulica': 'Blažov',
  'cislo': '58',
  'psc': '92901',
  'mesto': 'Kútniky'},
 'skNace': '46690',
 'typZavierky': 'Riadna',
 'obdobieOd': '2013-01',
 'obdobieDo': '2013-12',
 'predchadzajuceObdobieOd': '2012-05',
 'predchadzajuceObdobieDo': '2012-12',
 'datumZostavenia': '2014-02-28'}

{'sk': 'Strana aktív'}
['12164', '', '12164', '243764', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '12164', '', '12164', '242407', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '7008', '', '7008', '231125', '2500', '', '2500', '228361', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '4508', '', '4508', '2764', '', '', '', '', '5156', '', '5156', '11282', '5000', '',